# Simple System Manager - Parameter Store

The simple system manager has a parameter store to allow parameters to be stored and accessed. It can be secured using standard AWS security, can encrypt credentials, and so on.

In [ ]:
import boto3

ssm_client = boto3.client('ssm')

In [ ]:
# Store a value
response = ssm_client.put_parameter(
    Name='sample.p1',
    Description='Parameter p1 in sample namespace',
    Value='p1 value',
    Type='String',
    Overwrite=True
)

print response

In [ ]:
response = ssm_client.describe_parameters()
print response

In [ ]:
# Retrieve a value

response = ssm_client.get_parameters(
    Names=[
        'sample.p1',
    ]
)

params = response['Parameters']
print 'Value of sample.p1: \'{}\''.format(next((x['Value'] for x in params if x['Name'] == 'sample.p1'), None))
print 'Value of foo: {}'.format(next((x['Value'] for x in params if x['Name'] == 'foo'), None))